<a href="https://colab.research.google.com/github/bkvkrll/-Introduction-to-Natural-Language-Processing/blob/main/CW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot --upgrade

In [ ]:
!pip install pymorphy2

In [ ]:
!pip install stop_words

In [ ]:

!pip install annoy

In [ ]:
import numpy as np
import pandas as pd

import os
import annoy
import pickle
import string

from tqdm.notebook import tqdm_notebook
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from telegram.ext  import Updater, CommandHandler, MessageHandler, filters
from telegram import Animation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
products_dataset = pd.read_csv("/content/drive/My Drive/ProductsDataset.csv")
input_file = "/content/drive/My Drive/Otvety.txt"
output_file = "/content/drive/My Drive/prepared_answers.txt"

In [ ]:
def preprocess_txt(line):
    """Строковый препроцессинг текстовой строки"""
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [ ]:
assert True

#Small preprocess of the answers

question = None
written = False

with open(output_file, "w", encoding="utf8") as fout:
    with open(input_file, "r", encoding="utf8") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

In [ ]:
%%time

assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open(input_file, "r", encoding="utf8") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

In [ ]:
sentences = [i for i in sentences if len(i) > 2]
modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)
modelFT.save("ft_model")

In [ ]:
%%time

modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open(output_file, "r", encoding="utf8") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

ft_index.build(10)
ft_index.save('speaker.ann')

In [ ]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')

In [ ]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

In [ ]:
products_dataset['text'] = products_dataset['title'] + " " + products_dataset["descrirption"]
products_dataset['text'] = products_dataset['text'].apply(lambda x: preprocess_txt(str(x)))
products_dataset.head()

In [ ]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

In [ ]:
idxs = set(np.random.randint(0, len(index_map), len(products_dataset)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in products_dataset['text'].values]

In [ ]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, 
                                                    stratify=labels, random_state=13)

In [ ]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [ ]:
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

In [ ]:
assert True

with open("/content/drive/My Drive/Colab Notebooks/index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [ ]:
tfidf_vect = TfidfVectorizer().fit(X_train)

In [ ]:
np.mean(tfidf_vect.idf_)

In [ ]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

In [ ]:
ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(products_dataset))):
    n_ft = 0
    index_map_shop[counter] = (products_dataset.loc[i, "title"], products_dataset.loc[i, "image_links"])
    vector_ft = np.zeros(100)
    for word in products_dataset.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

In [ ]:
assert True

with open("/content/drive/My Drive/Colab Notebooks/index_shop.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [ ]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 1, include_distances=True)

In [ ]:
!pip install telegram

In [ ]:
!pip uninstall tornado
!pip install tornado==4.5.3

In [ ]:

!pip uninstall telegram==0.0.1 

In [ ]:
start_msg = 'Привет. Я бот.  Чем могу помочь?'

In [ ]:
updater = Updater(token='')
dispatcher = updater.dispatcher

In [ ]:
def startCommand(bot, update):
    bot.send_message(chat_id=update.message.chat_id, text=start_msg)

In [ ]:
def textMessage(bot, update):
    
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.2:
        print(distances[0])
        bot.send_message(chat_id=update.message.chat_id, text="Чаво ?")
        return
    bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])

In [ ]:
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()